# Accumulator Buffer

The accumulator buffer is needed to store partial results of large matrix multiplications from the outputs of the systolic array.

Each accumulator bank will connect to a column output from the systolic array. We can specify the address width which will determine the number of values we can store. This is equal to `2^len(mem_addr)`. For control signals to each